In [1]:
from collections import Counter 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re
import nltk
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk import bigrams
from tqdm import tqdm
# import string library function  
import string  
# natural language processing: n-gram ranking
import re
import unicodedata
from nltk.corpus import stopwords
# add appropriate words that will be ignored in the analysis
ADDITIONAL_STOPWORDS = ['covfefe']
import spacy
nlp = spacy.load('en_core_web_sm')
stopword = nltk.corpus.stopwords.words('english')

In [2]:
with open('intermediate_files/train_data_for_amazon_sarcasm_detection.pkl', "rb") as fh:
    dft = pickle.load(fh)
with open('intermediate_files/test_data_for_amazon_sarcasm_detection.pkl', "rb") as fh:
    dftest = pickle.load(fh)

# Training Part

## 1. Hyperbole Weight 

In [3]:
dft

,review,rating,label
0,I simply had no idea how bad it is! I am a sta...,5.0,0
1,A searing indictment... This book is sure to b...,5.0,0
2,Another movie to ignore.... A perfect date mov...,1.0,1
3,Buy this phone !! I got my Droid Incredible in...,5.0,0
4,Mother & daughter So far this is pretty boring...,3.0,0
...,...,...,...
647,Plays tracks in alphanumeric order This is a g...,1.0,0
648,be very CAREFUL!!!! This is good phone. It is ...,1.0,1
649,Really like this movie and its sequeal 10.5 ap...,5.0,0
650,Early Album of 2010 Candidate I open this revi...,5.0,0


In [4]:
def basic_clean(text):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    
    #return [wnl.lemmatize(word) for word in words if word not in stopwords]
    return [wnl.lemmatize(word) for word in words]



def extractPOS(df):
    Npos = []
    Tpos = []
    for d in df:
        doc = re.findall(r"([^.]*?[^.]*\.)",d)
        for n in doc:
            n=' '.join(basic_clean(n))
            for sj in nlp(n):
                    Npos.append((sj.pos_))
                    Tpos.append((sj.text))
    return (Npos,Tpos)


def hyper_base(train):
    n=2 #gram value
    #Whole Corpus hyperbole gram
    Npos,Tpos = extractPOS(train['review'])
    SbigramP=pd.Series(nltk.ngrams(Npos, n))
    TSbigramP=pd.Series(nltk.ngrams(Tpos, n))
    l=list(SbigramP)
    Tl=list(TSbigramP)
    TrainHyper=[]
    for i,j in zip(l,Tl):
        if i == ('ADV', 'ADJ'):
            TrainHyper.append(' '.join(j))

    dtf=pd.DataFrame(TrainHyper).value_counts()
    trainGram=pd.DataFrame([dtf.index.values,dtf.values]).transpose()

    #sarcastic hyperbole gram
    df=train[train['label']==1]
    Npos,Tpos = extractPOS(df['review'])
    SbigramP=pd.Series(nltk.ngrams(Npos, n))
    TSbigramP=pd.Series(nltk.ngrams(Tpos, n))
    l=list(SbigramP)
    Tl=list(TSbigramP)
    Shyper=[]
    for i,j in zip(l,Tl):
        if i == ('ADV', 'ADJ'):
            Shyper.append(' '.join(j))
    dtf=pd.DataFrame(Shyper).value_counts()
    SarcHyper=pd.DataFrame([dtf.index.values,dtf.values]).transpose()


    #Non-sarcastic hyperbole gram
    df=train[train['label']==0]
    Npos,Tpos = extractPOS(df['review'])
    SbigramP=pd.Series(nltk.ngrams(Npos, n))
    TSbigramP=pd.Series(nltk.ngrams(Tpos, n))
    l=list(SbigramP)
    Tl=list(TSbigramP)
    Nshyper=[]
    for i,j in zip(l,Tl):
        if i == ('ADV', 'ADJ'):
            Nshyper.append(' '.join(j))
    dtf=pd.DataFrame(Nshyper).value_counts()
    NsarcHyper=pd.DataFrame([dtf.index.values,dtf.values]).transpose()
    
    return SarcHyper,NsarcHyper,trainGram

In [5]:
def hyper_Wt(x,n,s,set_Hyper,SarcHyper,NsarcHyper,trainGram):
    if s==1:
        labelHyper=SarcHyper
    else:
        labelHyper=NsarcHyper
    
    #Document Sarcastic N-gram
    Npos,Tpos = extractPOS(x)
    SbigramP=pd.Series(nltk.ngrams(Npos, n))
    TSbigramP=pd.Series(nltk.ngrams(Tpos, n))
    l=list(SbigramP)
    Tl=list(TSbigramP)
    Dhyper=[]
    for i,j in zip(l,Tl):
        if i == ('ADV', 'ADJ'):
            Dhyper.append(' '.join(j))
    gram=[]
    DocHyper=pd.DataFrame(pd.DataFrame(Dhyper).value_counts())
    dtf=pd.DataFrame(Dhyper).value_counts()
    DocHyper=pd.DataFrame([dtf.index.values,dtf.values]).transpose()
    
    length=len(list(set(Dhyper)))
    if length==0:
        set_Hyper.append('')
    else:
        set_Hyper.append(list(set(Dhyper)))
        
    for i in range(len(DocHyper)):
        try:
            nGram =labelHyper[labelHyper[0]==DocHyper[0][i]][1].values[0]
        except:
            nGram=0
        gram.append(nGram)
   # print(gram)
    return (sum(gram)/sum(trainGram[1]))#/length

In [6]:
## for train hyperbole score
SarcHyper, NsarcHyper, trainGram = hyper_base(dft)
def hyper_df(dft):

    n=2
    hyperScore=[]
    set_Hyper=[]
    for i in range(len(dft)):
        res=hyper_Wt(dft.iloc[i:i+1,0],n,dft.iloc[i,2],set_Hyper,SarcHyper,NsarcHyper,trainGram)
        hyperScore.append(res)
    hyperScore=list(np.nan_to_num(hyperScore))
    df = dft.copy()
    df['hyper_set']=set_Hyper
    df['hyper_wt']=hyperScore
    
    return df

In [7]:
hyper_df1 = hyper_df(dft)

In [8]:
hyper_df1

,review,rating,label,hyper_set,hyper_wt
0,I simply had no idea how bad it is! I am a sta...,5.0,0,"[more corrupted, most liberal, how bad]",0.002737
1,A searing indictment... This book is sure to b...,5.0,0,"[entirely new, most damning, how lucky, veheme...",0.002737
2,Another movie to ignore.... A perfect date mov...,1.0,1,,0.000000
3,Buy this phone !! I got my Droid Incredible in...,5.0,0,"[super fast, soon bottom]",0.001095
4,Mother & daughter So far this is pretty boring...,3.0,0,[pretty boring],0.000547
...,...,...,...,...,...
647,Plays tracks in alphanumeric order This is a g...,1.0,0,,0.000000
648,be very CAREFUL!!!! This is good phone. It is ...,1.0,1,"[very careful, quite flimsy, very easy]",0.001642
649,Really like this movie and its sequeal 10.5 ap...,5.0,0,,0.000000
650,Early Album of 2010 Candidate I open this revi...,5.0,0,"[pretty good, seriously sick, more due, most s...",0.019704


## 2. Punctuation weight

In [9]:
def punc_df(dft):
    def punc_base(dft):
        Sdoc_punc=[]
        Ndoc_punc=[]
        for doc,l in zip(dft['review'],dft['label']):
            sent_punc=[]
            for t in doc:
                if t in '!"\'?.':
                    sent_punc.append(t)
            if l==1:
                Sdoc_punc.append(sent_punc)
            else:
                Ndoc_punc.append(sent_punc)

        all_punc=[]
        for i in Sdoc_punc+Ndoc_punc:
            all_punc=all_punc+i

        total_occ= sum(Counter(all_punc).values())

        return Sdoc_punc,Ndoc_punc,total_occ


    def punc_wt(doc_punc):
        temp_punc=[]
        for i in doc_punc:
            temp_punc=temp_punc+i

        wt=[]
        punc_set=[]
        for s in doc_punc:
            add=0
            for j in set(s):
                add=add+Counter(temp_punc)[j]
            wt.append(add/total_occ)
            punc_set.append(list(set(s)))
        return (wt,punc_set)


    Sdoc_punc,Ndoc_punc,total_occ = punc_base(dft)
    
    temp_df=dft[dft['label']==1]
    temp_df['punc_set']=punc_wt(Sdoc_punc)[1]
    temp_df['punc_wt']=punc_wt(Sdoc_punc)[0]

    temp_df1=dft[dft['label']==0]
    temp_df1['punc_set']=punc_wt(Ndoc_punc)[1]
    temp_df1['punc_wt']=punc_wt(Ndoc_punc)[0]

    df = pd.concat([temp_df,temp_df1])
    df=df.sort_index()
    return df

In [10]:
punc_df1 = punc_df(dft)

<ipython-input-9-5e0152d55608>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['punc_set']=punc_wt(Sdoc_punc)[1]
<ipython-input-9-5e0152d55608>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['punc_wt']=punc_wt(Sdoc_punc)[0]
<ipython-input-9-5e0152d55608>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [11]:
punc_df1

,review,rating,label,punc_set,punc_wt
0,I simply had no idea how bad it is! I am a sta...,5.0,0,"[., !, ', ""]",0.471721
1,A searing indictment... This book is sure to b...,5.0,0,"[., ', "", !]",0.471721
2,Another movie to ignore.... A perfect date mov...,1.0,1,"[., ']",0.412439
3,Buy this phone !! I got my Droid Incredible in...,5.0,0,"[., !, ', ""]",0.471721
4,Mother & daughter So far this is pretty boring...,3.0,0,"[?, ., ', ""]",0.452594
...,...,...,...,...,...
647,Plays tracks in alphanumeric order This is a g...,1.0,0,"[., ']",0.401862
648,be very CAREFUL!!!! This is good phone. It is ...,1.0,1,"[., !]",0.342390
649,Really like this movie and its sequeal 10.5 ap...,5.0,0,"[., ']",0.401862
650,Early Album of 2010 Candidate I open this revi...,5.0,0,"[., ', "", !]",0.471721


## 3. Interjection weight

In [12]:
def basic_clean(text):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    
    #return [wnl.lemmatize(word) for word in words if word not in stopwords]
    return [wnl.lemmatize(word) for word in words]


In [13]:
def intj_df(dft):
    
    def intj_base(dft):
        Sdoc_intj=[]
        Ndoc_intj=[]
        for d,l in zip(dft['review'],dft['label']):
            sent_intj=[]
            # Intj extract 
            doc = re.findall(r"([^.]*?[^.]*\.)",d)
            for n in doc:
                n=' '.join(basic_clean(n))
                for sj in nlp(n):
                    if sj.pos_ == 'INTJ':
                        sent_intj.append((sj.text))

            if l==1:
                Sdoc_intj.append(sent_intj)
            else:
                Ndoc_intj.append(sent_intj)


        all_intj=[]
        for i in Sdoc_intj+Ndoc_intj:
            all_intj=all_intj+i

        total_occ= sum(Counter(all_intj).values())
        
        return Sdoc_intj,Ndoc_intj, total_occ

    def intj_wt(doc_intj):
        temp_intj=[]
        for i in doc_intj:
            temp_intj=temp_intj+i

        wt=[]
        intj_set=[]
        for s in doc_intj:
            add=0
            for j in set(s):
                add=add+Counter(temp_intj)[j]
            wt.append(add/total_occ)
            intj_set.append(list(set(s)))
        return (wt,intj_set)
    
    Sdoc_intj,Ndoc_intj, total_occ = intj_base(dft)
    temp_df=dft[dft['label']==1]
    temp_df['intj_set']=intj_wt(Sdoc_intj)[1]
    temp_df['intj_wt']=intj_wt(Sdoc_intj)[0]

    temp_df1=dft[dft['label']==0]
    temp_df1['intj_set']=intj_wt(Ndoc_intj)[1]
    temp_df1['intj_wt']=intj_wt(Ndoc_intj)[0]

    df = pd.concat([temp_df,temp_df1])
    df=df.sort_index()
    return df

In [14]:
intj_df1 = intj_df(dft)

<ipython-input-13-c28bf1dcdfdb>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['intj_set']=intj_wt(Sdoc_intj)[1]
<ipython-input-13-c28bf1dcdfdb>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['intj_wt']=intj_wt(Sdoc_intj)[0]
<ipython-input-13-c28bf1dcdfdb>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [15]:
intj_df1

,review,rating,label,intj_set,intj_wt
0,I simply had no idea how bad it is! I am a sta...,5.0,0,"[blah, well]",0.040428
1,A searing indictment... This book is sure to b...,5.0,0,[],0.000000
2,Another movie to ignore.... A perfect date mov...,1.0,1,[],0.000000
3,Buy this phone !! I got my Droid Incredible in...,5.0,0,"[ha, like]",0.250892
4,Mother & daughter So far this is pretty boring...,3.0,0,[ha],0.229489
...,...,...,...,...,...
647,Plays tracks in alphanumeric order This is a g...,1.0,0,"[sure, nx]",0.005945
648,be very CAREFUL!!!! This is good phone. It is ...,1.0,1,[],0.000000
649,Really like this movie and its sequeal 10.5 ap...,5.0,0,[],0.000000
650,Early Album of 2010 Candidate I open this revi...,5.0,0,"[ha, please]",0.237812


In [16]:
wt_hyper_intj_puct = pd.merge(hyper_df1,punc_df1)
wt_hyper_intj_puct = pd.merge(wt_hyper_intj_puct,intj_df1)

In [17]:
wt_hyper_intj_puct.to_pickle('intermediate_files/Train_hyper_intj_puct_wt.pkl')

In [18]:
wt_hyper_intj_puct

,review,rating,label,hyper_set,hyper_wt,punc_set,punc_wt,intj_set,intj_wt
0,I simply had no idea how bad it is! I am a sta...,5.0,0,"[more corrupted, most liberal, how bad]",0.002737,"[., !, ', ""]",0.471721,"[blah, well]",0.040428
1,A searing indictment... This book is sure to b...,5.0,0,"[entirely new, most damning, how lucky, veheme...",0.002737,"[., ', "", !]",0.471721,[],0.000000
2,Another movie to ignore.... A perfect date mov...,1.0,1,,0.000000,"[., ']",0.412439,[],0.000000
3,Buy this phone !! I got my Droid Incredible in...,5.0,0,"[super fast, soon bottom]",0.001095,"[., !, ', ""]",0.471721,"[ha, like]",0.250892
4,Mother & daughter So far this is pretty boring...,3.0,0,[pretty boring],0.000547,"[?, ., ', ""]",0.452594,[ha],0.229489
...,...,...,...,...,...,...,...,...,...
647,Plays tracks in alphanumeric order This is a g...,1.0,0,,0.000000,"[., ']",0.401862,"[sure, nx]",0.005945
648,be very CAREFUL!!!! This is good phone. It is ...,1.0,1,"[very careful, quite flimsy, very easy]",0.001642,"[., !]",0.342390,[],0.000000
649,Really like this movie and its sequeal 10.5 ap...,5.0,0,,0.000000,"[., ']",0.401862,[],0.000000
650,Early Album of 2010 Candidate I open this revi...,5.0,0,"[pretty good, seriously sick, more due, most s...",0.019704,"[., ', "", !]",0.471721,"[ha, please]",0.237812


# Testing Part

## 1. Hyperbole Set 

In [19]:
def test_hyper(dftest):
    n=2
    t_hype=[]
    for j in range(0,2):
        hyperScore=[]
        set_Hyper=[]
        for i in range(len(dftest)):
            res=hyper_Wt(dftest.iloc[i:i+1,0],n,j,set_Hyper,SarcHyper,NsarcHyper,trainGram)
            
    return set_Hyper

## 2. Punctuation Set

In [20]:
def test_pucn(dftest):    
    TestDoc_punc=[]

    for doc in dftest['review']:
        sent_punc=[]
        for t in doc:
            if t in '!"\'?.':
                sent_punc.append(t)
        TestDoc_punc.append(list(set(sent_punc)))
    return TestDoc_punc

## 3. Interjection Set

In [21]:
def test_intj(dftest):
    TestDoc_intj=[]
    for d in dftest['review']:
        sent_intj=[]
        # Intj extract 
        doc = re.findall(r"([^.]*?[^.]*\.)",d)
        for n in doc:
            n=' '.join(basic_clean(n))
            for sj in nlp(n):
                if sj.pos_ == 'INTJ':
                    sent_intj.append((sj.text))

        TestDoc_intj.append(list(set(sent_intj)))
    
    return TestDoc_intj

In [22]:
dftest['hyper_set'] = test_hyper(dftest)
dftest['punc_set'] = test_pucn(dftest)
dftest['intj_set'] = test_intj(dftest)

In [23]:
dftest

,review,rating,label,hyper_set,punc_set,intj_set
0,"Great Alyssa, but rather boring movie This is ...",2.0,0,"[very little, very sexy, really good]","[., ']",[]
1,worst ever If this was written by Danielle Ste...,1.0,1,"[so bad, very least, how bad]","[., !, ']",[]
2,Hasselhoff Me! Please! One was having a parti...,5.0,1,"[particularly good, quite refreshed, particula...","[', !, ?, ., ""]","[ha, please]"
3,"One Friday, Without the Milk He always brought...",3.0,1,,"[?, ., ', ""]","[oz, wa, well]"
4,"this. sucked. bad. one word: punctuation. ok, ...",1.0,1,"[thuroughly impossible, mildly understandable]","[?, ., !]","[ok, oh, like]"
...,...,...,...,...,...,...
213,"If dragons could wear t-shirts, this is the on...",5.0,1,[currently impossible],"[., ', ""]",[like]
214,Horrible Product this product only got one sta...,1.0,1,,"[., ', "", !]",[]
215,BDP-S560 vs Panasonic DMP-BD80 There seems to ...,5.0,0,"[extremely careful, very deep, too cheap, more...","[', !, ?, ., ""]","[right, ha, anyway]"
216,Great Gift... I know nothing about guitars so ...,4.0,0,,"[., !]",[ha]


In [24]:
dftest.to_pickle('intermediate_files/Test_hyper_intj_puct.pkl')